In [1]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore') 
dataset = pd.read_csv("/kaggle/input/d/omarnasr/ai-image-classification-2/image_labels.tsv",sep="\t")

In [2]:
!git clone https://github.com/Omar-Nasr/ai_generated_image_classification_defactify_2024

Cloning into 'ai_generated_image_classification_defactify_2024'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 202 (delta 26), reused 18 (delta 7), pack-reused 156 (from 1)
Receiving objects: 100% (202/202), 171.54 KiB | 1.28 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [3]:
import sys
sys.path.append("/kaggle/working/ai_generated_image_classification_defactify_2024")

In [4]:
!cd /kaggle/working/ai_generated_image_classification_defactify_2024 && git pull

Already up to date.


In [5]:
from data_handler import Image_Classification_Dataset

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [6]:
!mkdir checkpoints

In [7]:
train_data_dir = "/kaggle/input/d/omarnasr/ai-image-classification-2/" 
checkpoint_path = "checkpoints/"
val_data_dir = "/kaggle/input/ai-image-classification-val-test/"

In [8]:
!curl "https://drive.usercontent.google.com/download?id=1mutiXZZGP9UhtqOLSDspJHX7VNafT2z0&export=download&authuser=0&confirm=t&uuid=85b29666-6ba1-4a29-830c-7bc011d7a176&at=AENtkXb0JST3mpXXQql8s83jTM4M:1730885596175" --output val.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  226k  100  226k    0     0  73071      0  0:00:03  0:00:03 --:--:-- 73078


In [9]:
!unzip -P DEFACTIFY-4_IMG_VAL "/kaggle/working/val.zip"

Archive:  /kaggle/working/val.zip
   creating: val_dataset/
  inflating: val_dataset/Readme_Val_data.md  
  inflating: val_dataset/Validation_labels.xlsx  


In [10]:
val_labels_df = pd.read_excel("/kaggle/working/val_dataset/Validation_labels.xlsx")
val_labels_df

,Index,Caption,Label_A,Label_B
0,0,a toilet sits next to a shower an sink,1,3
1,1,A TV sitting on top of a table next to a lapto...,0,0
2,2,Two giraffes eat from a pot attached to a fence.,1,1
3,3,The kitchen has many grill with pots hanging a...,1,4
4,4,some road signs besides a road in the street,1,1
...,...,...,...,...
8995,8995,Three red double decker buses parked side by s...,0,0
8996,8996,A white toilet in a white tiled bathroom.,1,3
8997,8997,A bus is stopping to pick up a passenger.,0,0
8998,8998,Small birds in the grass in the sun,1,4


In [11]:

val_labels = np.array(val_labels_df.iloc[:,2])
val_multiclass_labels = np.array(val_labels_df.iloc[:,3])
val_multiclass_labels

array([3, 0, 1, ..., 0, 4, 2])

In [12]:
from data_handler import Image_Classification_Dataset
from torch.utils.data import DataLoader
val = Image_Classification_Dataset(val_data_dir,task="Binary",val=True,val_labels=val_labels)
test = Image_Classification_Dataset(val_data_dir,task="Binary",val=False,test=True)
test_loader = DataLoader(test,16,num_workers=4)

In [13]:
from parameter_tuning import objective
from functools import partial
from optuna.trial import TrialState
import optuna 
dict
study = optuna.create_study(direction="maximize")
objective = partial(objective, train_data_dir=train_data_dir,checkpoint_path=checkpoint_path,num_epochs=2,val_data_dir=val_data_dir,val_labels = val_multiclass_labels,val=True,task="Multiclass",optimizer_name="adam",batch_sz=16)
study.optimize(objective, n_trials=5)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
with open("bestparams","w") as f:
    for key, value in trial.params.items():
        f.write("    {}: {}".format(key, value))


[I 2024-11-29 13:07:42,509] A new study created in memory with name: no-name-8b8770fd-ca07-41dd-98fb-628323244fd4
Downloading: "https://download.pytorch.org/models/swin_v2_b-781e5279.pth" to /root/.cache/torch/hub/checkpoints/swin_v2_b-781e5279.pth
100%|██████████| 336M/336M [00:01<00:00, 236MB/s]


Epoch 0/1

----------



libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

Epoch $0 Training Loss: $32778.76197504997 

Epoch $0 Training F1_Score 0.7100476026535034 Best F1: 0.7100476026535034


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


...

Epoch $1 Training Loss: $63978.02671432495 

Epoch $1 Training F1_Score 0.42461904883384705 Best F1: 0.42461904883384705


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 1000000h: invalid rendering intent
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch $1 Val Loss: $13261.824257850647 

Epoch $1 Val F1_Score 0.4122222363948822 Best F1: 0.4122222363948822
Training complete in 41m 22s

Best F1: 0.4122222363948822 after 2 Epochs


[I 2024-11-29 16:34:45,932] Trial 4 finished with value: 0.4122222363948822 and parameters: {'lr': 1.2261199563277276e-07, 'lr2': 4.765787643138384e-06}. Best is trial 0 with value: 0.526888906955719.


Study statistics: 
  Number of finished trials:  5
  Number of pruned trials:  0
  Number of complete trials:  5
Best trial:
  Value:  0.526888906955719
  Params: 


In [14]:
# from models import train_classifier
# model_adopt_binary,classifier_adopt = train_classifier(train_data_dir,checkpoint_path,num_epochs=3,val_data_dir=val_data_dir,val_labels = val_labels,val=True,use_fourier=False,optimizer_name="adopt",lr=1e-5,fine_tune=False)

In [15]:
# from models import train_classifier
# model_adopt_multiclass,classifier_adopt_multiclass = train_classifier(train_data_dir,checkpoint_path,num_epochs=5,val_data_dir=val_data_dir,val_labels = val_multiclass_labels,val=True,task="Multiclass",use_fourier=False,optimizer_name="adopt",lr=1e-5,fine_tune=False)

In [16]:
# from models import train_classifier
# model_adam_binary,classifier_adam = train_classifier(train_data_dir,checkpoint_path,num_epochs=3,val_data_dir=val_data_dir,val_labels = val_labels,val=True,use_fourier=False)
# model_adam_multiclass,classifier_adam_multiclass = train_classifier(train_data_dir,checkpoint_path,num_epochs=3,val_data_dir=val_data_dir,val_labels = val_multiclass_labels,val=True,task="Multiclass",use_fourier=False)

In [17]:
from torchvision import models
# import torch
# device = xm.xla_device()
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model_adopt_binary.to(device)
# classifier_adopt.to(device)
# model_adopt_multiclass.to(device)
# classifier_adopt_multiclass.to(device)
# model_adam_multiclass.to(device)
# classifier_adam_multiclass.to(device)
# test_preds_adam_binary = []
# test_preds_adam_multiclass = []
# test_preds_adopt_binary = []
# test_preds_adopt_multiclass = []
# for inputs in test_loader:
#         inputs = inputs.to(device)
#         features = model_adam_multiclass(inputs)
#         outputs = classifier_adam_multiclass(features)
#         _,preds = torch.max(outputs,1)
#         test_preds_adam_multiclass.append(preds.cpu())
# for inputs in test_loader:
#         inputs = inputs.to(device)
#         features = model_adam_binary(inputs)
#         outputs = classifier_adam(features)
#         _,preds = torch.max(outputs,1)
        # test_preds_adam_binary.append(preds.cpu())    
# for inputs in test_loader:
#         inputs = inputs.to(device)
#         features = model_adopt_multiclass(inputs)
#         outputs = classifier_adopt_multiclass(features)
#         _,preds = torch.max(outputs,1)
#         test_preds_adopt_multiclass.append(preds.cpu())
# for inputs in test_loader:
#         inputs = inputs.to(device)
#         features = model_adopt_binary(inputs)
#         outputs = classifier_adopt(features)
#         _,preds = torch.max(outputs,1)
#         test_preds_adopt_binary.append(preds.cpu())    
# test_preds_adopt_binary = np.concatenate(test_preds_adopt_binary)
# test_preds_adopt_multiclass = np.concatenate(test_preds_adopt_multiclass)
# test_preds_adam_binary = np.concatenate(test_preds_adam_binary)
# test_preds_adam_multiclass = np.concatenate(test_preds_adam_multiclass)


In [18]:
# from datasets import load_dataset
# import json
# ds = load_dataset("NasrinImp/Defactify4_Test")
# captions = ds["train"]['caption']
# list = []
# for i,pred in enumerate(test_preds_adam_binary):
#     dict = {}
#     dict["index"]=i
#     dict["caption"]=captions[i]
#     dict["Label_A"]=int(test_preds_adam_binary[i])
#     dict["Label_B"]=int(test_preds_adam_multiclass[i])
#     list.append(dict)
# json_object = json.dumps(list,indent=4)
# with open("answer_adam.json","w") as f:
#     f.write(json_object)

In [19]:
# from datasets import load_dataset
# import json
# ds = load_dataset("NasrinImp/Defactify4_Test")
# captions = ds["train"]['caption']
# list = []
# for i,pred in enumerate(test_preds_adopt_binary):
#     dict = {}
#     dict["index"]=i
#     dict["caption"]=captions[i]
#     dict["Label_A"]=int(test_preds_adopt_binary[i])
#     dict["Label_B"]=int(test_preds_adopt_multiclass[i])
#     list.append(dict)
# json_object = json.dumps(list,indent=4)
# with open("answer_adopt.json","w") as f:
#     f.write(json_object)